# PyTerrier Notebook for Full-Rank Submissions

This notebook serves as a baseline full-rank submission for [TIRA](https://tira.io)/[TIREx](https://tira.io/tirex) that builds a PyTerrier index and subsequently creates a run with BM25.

### Step 1: Ensure Libraries are Imported

In [1]:
import os

# Detect if we are in the TIRA sandbox
# Install the required dependencies if we are not in the sandbox.
if 'TIRA_DATASET_ID' not in os.environ:
    !pip3 install  python-terrier tira==0.0.88 ir_datasets
    !pip3 install -q python-terrier
    !pip3 install -q --upgrade git+https://github.com/terrierteam/pyterrier_t5.git
else:
    print('We are in the TIRA sandbox.')

from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run

# PyTerrier must be imported after the call to ensure_pyterrier_is_loaded in TIRA.
import pyterrier as pt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 2.3 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.1 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 1.5 MB/s eta 0:00:00ta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 690.4 kB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 988.7 kB/s eta 0:00:00a 0:00:01
     ━━━━━

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Ensure Pyterrier integration is loaded

In [2]:
ensure_pyterrier_is_loaded()

Due to execution in TIRA, I have patched ir_datasets to always return the single input dataset mounted to the sandbox.
Start PyTerrier with version=5.7, helper_version=0.0.7, no_download=True
terrier-assemblies 5.7 jar-with-dependencies not found, downloading to /home/codespace/.pyterrier...
Done
terrier-python-helper 0.0.7 jar not found, downloading to /home/codespace/.pyterrier...
Done
terrier-prf -SNAPSHOT jar not found, downloading to /home/codespace/.pyterrier...
Done


PyTerrier 0.10.0 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


### Step 2: Load data, create index

In [8]:
dataset = pt.get_dataset('irds:ir-lab-jena-leipzig-wise-2023/validation-20231104-training')
qrels = dataset.get_qrels()
topics = dataset.get_topics(variant="title")[:5]

index_loc = "./index"
indexer = pt.IterDictIndexer(index_loc)
indexref = indexer.index(dataset.get_corpus_iter())

No settings given in /home/codespace/.tira/.tira-settings.json. I will use defaults.
No settings given in /home/codespace/.tira/.tira-settings.json. I will use defaults.
File is not a zip file
Code: 504
Error occured while fetching https://www.tira.io/data-download/training/input-truth/validation-20231104-training.zip. I will sleep 8 seconds and continue.
File is not a zip file
Code: 504
Error occured while fetching https://www.tira.io/data-download/training/input-truth/validation-20231104-training.zip. I will sleep 14 seconds and continue.
File is not a zip file
Code: 504
Error occured while fetching https://www.tira.io/data-download/training/input-truth/validation-20231104-training.zip. I will sleep 8 seconds and continue.
File is not a zip file
Code: 504
Error occured while fetching https://www.tira.io/data-download/training/input-truth/validation-20231104-training.zip. I will sleep 12 seconds and continue.
File is not a zip file
Code: 504
Error occured while fetching https://www.ti

FileNotFoundError: [Errno 2] No such file or directory: '/home/codespace/.tira/extracted_datasets/ir-lab-jena-leipzig-wise-2023/validation-20231104-training//validation-20231104-training' -> '/home/codespace/.tira/extracted_datasets/ir-lab-jena-leipzig-wise-2023/validation-20231104-training/truth-data'

### Step 3: Create retrieval pipeline

#### We aim for retrieving docs via a linear combination of PL2 and BM25. Firstly, let's focus on PL2.

In [4]:
pl2 = pt.BatchRetrieve(indexer, wmodel="PL2", verbose=True)

#### Next, we perform BM25-retrieval with query expansion.

In [5]:
bm25 = pt.BatchRetrieve(indexer, wmodel="BM25", verbose=True)

bo1_expansion = ~bm25 >> pt.rewrite.Bo1QueryExpansion(indexer)
bm25_bo1 = bo1_expansion >> bm25

#### Let's combine the two systems.

In [13]:
#bm25_bo1_pl2 = (2* bm25_bo1 + pl2).transform(topics)
bm25_bo1_pl2 = (2* bm25_bo1 + pl2)

#### Test to ensure that renaming of dataframe is needed for reranking.

In [21]:
run = bm25_bo1_pl2(topics)
run.rename(columns= {"query": "query_0", "query_0": "query"})

BR(PL2): 100%|██████████| 882/882 [00:17<00:00, 50.67q/s]


,qid,docid,docno,score,query,query_0,rank
0,q072210025,21318.0,doc072207501000,66.759043,recipe spring roll,applypipeline:off recip^1.053565089 spring^1.3...,1
1,q072210025,9672.0,doc072212607743,66.977549,recipe spring roll,applypipeline:off recip^1.053565089 spring^1.3...,0
2,q072210025,43796.0,doc072207504499,66.178928,recipe spring roll,applypipeline:off recip^1.053565089 spring^1.3...,2
3,q072210025,8216.0,doc072201202671,62.435225,recipe spring roll,applypipeline:off recip^1.053565089 spring^1.3...,3
4,q072210025,59542.0,doc072204307357,62.309674,recipe spring roll,applypipeline:off recip^1.053565089 spring^1.3...,4
...,...,...,...,...,...,...,...
1099732,q072230074,NaN,doc072211000339,3.473103,NaN,NaN,1309
1099733,q072230074,NaN,doc072208407385,3.473081,NaN,NaN,1310
1099734,q072230074,NaN,doc072203309049,3.472435,NaN,NaN,1311
1099735,q072230074,NaN,doc072202201538,3.471925,NaN,NaN,1312


### Next, we want to rerank the output with a transformer.

In [7]:
from pyterrier_t5 import MonoT5ReRanker
monoT5 = MonoT5ReRanker()

spiece.model: 100%|██████████| 792k/792k [00:00<00:00, 8.80MB/s]
tokenizer.json: 100%|██████████| 1.39M/1.39M [00:00<00:00, 13.0MB/s]
config.json: 100%|██████████| 1.21k/1.21k [00:00<00:00, 6.61MB/s]
/home/codespace/.python/current/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default l

In [22]:
import pandas as pd

corpus = pd.DataFrame(dataset.get_corpus_iter())


class GetText(pt.Transformer):
    def transform(self, topics_or_res: pd.DataFrame) -> pd.DataFrame:
        return pd.merge(topics_or_res, corpus, on="docno")
    
class ResetQueryColumn(pt.Transformer):
    def transform(self, topics_or_res: pd.DataFrame) -> pd.DataFrame:
        return topics_or_res.rename(columns= {"query": "query_0", "query_0": "query"})

No settings given in /home/codespace/.tira/.tira-settings.json. I will use defaults.
No settings given in /home/codespace/.tira/.tira-settings.json. I will use defaults.


ir-lab-jena-leipzig-wise-2023/validation-20231104-training documents:   3%|▎         | 1624/61307 [00:00<00:03, 16228.37it/s]

No settings given in /home/codespace/.tira/.tira-settings.json. I will use defaults.
No settings given in /home/codespace/.tira/.tira-settings.json. I will use defaults.


ir-lab-jena-leipzig-wise-2023/validation-20231104-training documents: 100%|██████████| 61307/61307 [00:03<00:00, 17158.84it/s]


In [9]:
from pyterrier_t5 import T5Tokenizer

#### Cache the pipeline.

In [ ]:
bm25_bo1_pl2_mono = ~(bm25_bo1_pl2 % 10 >> GetText() >> ResetQueryColumn()
        >> pt.text.sliding(length=400, stride=64, prepend_attr=None, text_attr="text")%5
        >> monoT5 )

### Hypothesis 1: There is a significant ($\alpha < 0.05$) difference w.r.t. nDCG between aggregating with max passage and mean passage.

#### Firstly, rerank with max passage aggregation.

In [23]:
bm25_bo1_pl2_max = (bm25_bo1_pl2_mono >> pt.text.max_passage()) 

#### Secondly, rerank with mean passage aggregation.

In [24]:
bm25_bo1_pl2_mean = (bm25_bo1_pl2_mono  >> pt.text.mean_passage()) 

#### Let's compare both systems.

In [25]:
pt.Experiment(
    [bm25_bo1_pl2_max(topics("text")), bm25_bo1_pl2_mean(topics("text"))],
    topics,
    qrels,
    eval_metrics=["ndcg"],
    names=["max passage", "mean passage"],
)

BR(PL2): 100%|██████████| 882/882 [00:17<00:00, 51.54q/s]


calling sliding on df of 8780 rows


monoT5:   0%|          | 2/18445 [00:15<40:24:10,  7.89s/batches]


KeyboardInterrupt: 

### Hypothesis 2: Choosing $k \in \{5\cdot i \mid i \in [1, 10]\}$ such that the nDCG-score of $k$-max average aggregation is maximized, yields a significantly ($\alpha < 0.05$)  better nDCG-score than using max passage or mean passage aggregation.

In [ ]:
bm25_bo1_pl2_kmax = (bm25_bo1_pl2 % 10 >> GetText() >> ResetQueryColumn()
        >> pt.text.sliding(length=400, stride=64, prepend_attr=None, text_attr="text")
        >> monoT5 
        >> pt.text.kmaxavg_passage(controls={'k' : 5})) 

#### Find $k$ such that nDCG of reranking with $k$-max average passage is maximized.

In [ ]:
pt.GridSearch(
    bm25_bo1_pl2_kmax,
    {bm25_bo1_pl2_kmax :  {'k' : [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]}},
    topics,
    qrels,
    'ndcg',
    verbose=True
)

In [16]:
print('Create max-run')
run_max = bm25_bo1_pl2_max(dataset.get_topics("text"))
print('Max-run was created')
print('Create mean-run')
run_mean = bm25_bo1_pl2_mean(dataset.get_topics("text"))
print('Done, mean-run was created')

Create max-run


BR(PL2): 100%|██████████| 882/882 [00:17<00:00, 49.01q/s]


calling sliding on df of 8780 rows


monoT5:   0%|          | 46/18445 [04:28<29:52:09,  5.84s/batches]


KeyboardInterrupt: 

### Step 4: Persist run.

In [8]:
persist_and_normalize_run(run_max, output_file="./max_output", system_name='t5-reranker')
persist_and_normalize_run(run_mean, output_file="./mean_output", system_name='t5-reranker')

I use the environment variable "TIRA_OUTPUT_DIR" to determine where I should store the run file using "." as default.
Done. run file is stored under "./run.txt".
